04 Convolution・Poolingレイヤの実装
==============================

* ここでは、`畳み込み層`と`プーリング層`を、Pythonで実装する

* ここで実装するクラスについても、`forward`と`backward`というメソッドを持たせ、モジュールとして利用できるように実装する

## 1.4次元配列

* `CNN`では、各層を流れるデータは4次元のデータ

    * 例)データ形状が$(10,1,28,28)$の場合、高さ：$28$、横幅：$28$、1チャンネルのデータ：$10$個
    
    * これをPythonで実装すると、以下のようになる

In [1]:
import numpy as np

x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [2]:
# 一つ目のデータにアクセス
x[0].shape

(1, 28, 28)

In [3]:
# 二つ目のデータにアクセス
x[1].shape

(1, 28, 28)

In [4]:
# 一つ目のデータの1チャンネル目の空間データにアクセスする
x[0, 0]

array([[0.32077928, 0.29080679, 0.34956036, 0.73591068, 0.53751238,
        0.80869238, 0.41161638, 0.38333609, 0.93993802, 0.25336515,
        0.6500292 , 0.50227037, 0.05040314, 0.50738197, 0.56090997,
        0.16993949, 0.61975273, 0.90157673, 0.7970026 , 0.98708644,
        0.24559619, 0.43968054, 0.80489307, 0.85735676, 0.20839634,
        0.64548453, 0.41795365, 0.07227652],
       [0.91348153, 0.92192911, 0.55481471, 0.97639028, 0.8090941 ,
        0.45286902, 0.77269812, 0.45722385, 0.43345731, 0.8514872 ,
        0.25946323, 0.38254151, 0.03033936, 0.84434613, 0.33161504,
        0.78234835, 0.17860317, 0.61979484, 0.75371293, 0.73333025,
        0.4960291 , 0.72509692, 0.13546208, 0.52445573, 0.43789528,
        0.34903377, 0.27861226, 0.45362705],
       [0.48559189, 0.12061816, 0.90365737, 0.80489654, 0.24669443,
        0.5207295 , 0.68980289, 0.07619061, 0.65713466, 0.18780821,
        0.11535532, 0.10706799, 0.36525884, 0.14885082, 0.17760495,
        0.49773503, 0.2938

In [5]:
x[0][0]

array([[0.32077928, 0.29080679, 0.34956036, 0.73591068, 0.53751238,
        0.80869238, 0.41161638, 0.38333609, 0.93993802, 0.25336515,
        0.6500292 , 0.50227037, 0.05040314, 0.50738197, 0.56090997,
        0.16993949, 0.61975273, 0.90157673, 0.7970026 , 0.98708644,
        0.24559619, 0.43968054, 0.80489307, 0.85735676, 0.20839634,
        0.64548453, 0.41795365, 0.07227652],
       [0.91348153, 0.92192911, 0.55481471, 0.97639028, 0.8090941 ,
        0.45286902, 0.77269812, 0.45722385, 0.43345731, 0.8514872 ,
        0.25946323, 0.38254151, 0.03033936, 0.84434613, 0.33161504,
        0.78234835, 0.17860317, 0.61979484, 0.75371293, 0.73333025,
        0.4960291 , 0.72509692, 0.13546208, 0.52445573, 0.43789528,
        0.34903377, 0.27861226, 0.45362705],
       [0.48559189, 0.12061816, 0.90365737, 0.80489654, 0.24669443,
        0.5207295 , 0.68980289, 0.07619061, 0.65713466, 0.18780821,
        0.11535532, 0.10706799, 0.36525884, 0.14885082, 0.17760495,
        0.49773503, 0.2938

* このように、CNNでは4次元のデータを扱うことになる

* ここでは、`im2col`というトリックによって、問題は簡単になる

## 2.im2colによる展開

* `im2col`：フィルター(重み)にとって都合の良いように入力データを展開する関数

    * 以下の図に示すように、入力データに対してフィルターを適用する場所の領域(3次元のブロック)を、横方向に1列に展開する
    
    * この展開処理を、フィルターを適用する全ての場所で行うのが`im2col`
   
![im2colの概略図](./images/im2colの概略図.png)

![フィルターの適用領域を先頭から順番に1列に展開する](./images/フィルターの適用領域を先頭から順番に1列に展開する.png)

* 実際の畳み込み演算の場合は、フィルター領域が重なる場合がほとんど

* フィルターの適用領域が重なる場合、`im2col`によって展開すると、展開後の要素の数は元のブロックの要素数よりも多くなる

    * そのため、`im2col`を使った実装では通常よりも多くのメモリを消費するという欠点がある
   

* しかし、大きな行列にまとめて計算することは、コンピュータで計算する上で多くの恩恵がある

    * 例)行列計算のライブラリなどは、行列の計算速度が高度に最適化されており、大きな行列の掛け算を高速に行うことができる
    
    * そのため、行列の計算に帰着させることで、線形代数ライブラリを有効に活用することができる
    


* `im2col`によって入力データを展開したら、畳み込み層のフィルター(重み)を1列に展開して、2つの行列の積を計算する

    * これは、全結合層のAffineレイヤで行なったことと同じ

![畳み込み演算のフィルター処理の詳細](./images/畳み込み演算のフィルター処理の詳細.png)

* `im2col`方式による出力結果は2次元の行列

    * CNNの場合、データは4次元配列として格納するので、2次元の出力データを適切な形状に整形する

## 3.Convolutionレイヤの実装

* `im2col`という関数は、ブラックボックスとして利用することを想定する

* この関数は、以下のインターフェースを持つ
    
    * `input_data`：$(データ数, チャンネル, 高さ, 横幅)$の4次元配列からなる入力データ
    
    * `filter_h`：フィルターの高さ
    
    * `filter_w`：フィルターの横幅
    
    * `stride`：ストライド
    
    * `pad`：パディング

* この`im2col`は、「フィルターサイズ」「ストライド」「パディング」を考慮して、入力データを2次元配列に展開する

In [6]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [7]:
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


* 1つ目の例は、バッチサイズが`1`で、チャンネル`3`の$7 \times 7$のデータ

* 2つ目の例は、バッチサイズが`10`で、データの形状は1つ目の例と同じ

* それぞれ`im2col`関数を適用すると、2次元目の要素数は`75`になる

* `im2col`を用いて、畳み込み層を実装する

In [8]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # フィルターの展開
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

* `Convolution`レイヤの初期化メソッドは、「フィルター(重み)」、「バイアス」、「ストライド」、「パディング」を引数として受け取る

    * フィルターは、`(FN, C, FH, FW)`の4次元の形状

* `im2col`は、入力データを展開し、フィルターも`reshape`を使って2次元配列に展開する

    * そして、その展開した行列の積を計算する

* フィルターの展開を行う箇所では、各フィルターのブロックを1列に展開して並べる

    * ここで、`reshape(FN, -1)`の`-1`は、多次元配列の要素数のつじつまが合うように要素数をまとめる
    
    * 例)$(10,3,5,5)$の形状の配列は、要素数が`750`個あるが、ここで`reshape(10, -1)`とすると、$(10, 75)$の形状の配列に整形される

* `forward`の実装では、最後に、出力サイズを適切な形状に整形する

    * 整形の際に、NumPyの`transpose`という関数を使う
    
    * これは、多次元配列の軸の順番を入れ替える関数
    
    * 以下の図のように、`0`から始まるインデックス(番号)の並びを指定することで、軸の順番を変更する
    
![numpyのtransposeによる軸の順番の入れ替え](./images/numpyのtransposeによる軸の順番の入れ替え.png)

* `im2col`によって展開することで、全結合層のAffineレイヤとほとんど同じように実装することができる

* `Convolution`レイヤの逆伝播の実装だが、Affineレイヤと同じように実装することができる

## 4.Poolingレイヤの実装

* Poolingレイヤの実装も、`im2col`を用いて入力データを展開する

    * ただし、プーリングの場合は、チャンネル方向には独立である点が畳み込み層の場合と異なる
    
    * 具体的には、以下の図に示すように、プーリングの適用領域はチャンネルごとに独立して展開する
    
![入力データに対してプーリング適用領域を展開](./images/入力データに対してプーリング適用領域を展開.png)

* 一度このように展開することで、あとは行列に対して、行ごとに最大値を求め、適切な形状に整形する

In [9]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 展開(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        # 最大値(2)
        out = np.max(col, axis=1)
        
        # 整形(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

### Poolingレイヤの実装の流れ

1. 入力データを展開する

1. 行ごとに最大値を求める

1. 適切な出力サイズに変形する

![Poolingレイヤの流れ](./images/Poolingレイヤの流れ.png)

* Poolingレイヤの`forward`処理では、入力データを、プーリングを行いやすい形に展開してしまえば、シンプルとなる

* 逆伝播は省略する

| 版   | 年/月/日   |
| ---- | ---------- |
| 初版 | 2019/05/15 |